# Advanced Analysis

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/monteplan/blob/main/notebooks/03_advanced_analysis.ipynb)

This notebook covers power-user features: alternative return models, stress testing, sensitivity analysis, and advanced metrics.

In [ ]:
# Uncomment and run in Google Colab:
# !pip install -q "monteplan @ git+https://github.com/engineerinvestor/monteplan.git" plotly

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

try:
    import plotly.graph_objects as go
    HAS_PLOTLY = True
except ImportError:
    HAS_PLOTLY = False
    print("plotly not installed -- falling back to matplotlib for interactive charts")

from monteplan.config.defaults import default_plan, default_market, default_policies, default_sim_config
from monteplan.config.schema import (
    MarketAssumptions, AssetClass, SimulationConfig, StressScenario,
    RegimeSwitchingConfig, RegimeConfig, PolicyBundle,
)
from monteplan.core.engine import simulate

In [ ]:
plan = default_plan()
market = default_market()
policies = default_policies()
sim_config = SimulationConfig(n_paths=1000, seed=42)

## Part 1: Return Models

### MVN vs Student-t

In [ ]:
# MVN (default)
result_mvn = simulate(plan, market, policies, sim_config)

# Student-t with df=5 (heavier tails)
market_t = market.model_copy(update={"return_model": "student_t", "degrees_of_freedom": 5.0})
result_t = simulate(plan, market_t, policies, sim_config)

print(f"MVN success:       {result_mvn.success_probability:.1%}")
print(f"Student-t success: {result_t.success_probability:.1%}")
print(f"\nTerminal wealth P5:")
print(f"  MVN:       ${result_mvn.terminal_wealth_percentiles['p5']:,.0f}")
print(f"  Student-t: ${result_t.terminal_wealth_percentiles['p5']:,.0f}")

### Regime Switching

In [ ]:
regime_config = RegimeSwitchingConfig(
    regimes=[
        RegimeConfig(
            name="bull",
            expected_annual_returns=[0.12, 0.05],
            annual_volatilities=[0.12, 0.04],
            correlation_matrix=[[1.0, -0.1], [-0.1, 1.0]],
            inflation_mean=0.025,
            inflation_vol=0.008,
        ),
        RegimeConfig(
            name="normal",
            expected_annual_returns=[0.07, 0.03],
            annual_volatilities=[0.16, 0.06],
            correlation_matrix=[[1.0, 0.0], [0.0, 1.0]],
            inflation_mean=0.03,
            inflation_vol=0.01,
        ),
        RegimeConfig(
            name="bear",
            expected_annual_returns=[-0.05, 0.01],
            annual_volatilities=[0.25, 0.08],
            correlation_matrix=[[1.0, 0.3], [0.3, 1.0]],
            inflation_mean=0.04,
            inflation_vol=0.02,
        ),
    ],
    transition_matrix=[
        [0.95, 0.04, 0.01],
        [0.05, 0.90, 0.05],
        [0.05, 0.10, 0.85],
    ],
    initial_regime=1,
)

market_regime = market.model_copy(update={
    "return_model": "regime_switching",
    "regime_switching": regime_config,
})
result_regime = simulate(plan, market_regime, policies, sim_config)

print(f"MVN success:            {result_mvn.success_probability:.1%}")
print(f"Regime switching success: {result_regime.success_probability:.1%}")

## Part 2: Stress Testing

In [ ]:
# Define stress scenarios
scenarios = {
    "Base (no stress)": [],
    "Crash at retirement": [
        StressScenario(name="Crash", scenario_type="crash", start_age=65, duration_months=12),
    ],
    "Sequence risk": [
        StressScenario(name="Sequence", scenario_type="sequence_risk", start_age=65, duration_months=36),
    ],
    "High inflation": [
        StressScenario(name="Inflation", scenario_type="high_inflation", start_age=65, duration_months=60),
    ],
}

stress_results = {}
for name, stress_list in scenarios.items():
    sc = SimulationConfig(n_paths=1000, seed=42, stress_scenarios=stress_list)
    res = simulate(plan, market, policies, sc)
    stress_results[name] = res
    print(f"{name:25s} -> Success: {res.success_probability:.1%}")

In [ ]:
# Compare stress scenarios
fig, ax = plt.subplots(figsize=(10, 5))
colors = ["#2ca02c", "#d62728", "#ff7f0e", "#9467bd"]
for (name, res), color in zip(stress_results.items(), colors):
    ts = res.wealth_time_series
    ages = np.linspace(plan.current_age, plan.end_age, len(ts["p50"]))
    ax.plot(ages, ts["p50"], label=f"{name} ({res.success_probability:.0%})", color=color, linewidth=2)

ax.axvline(plan.retirement_age, color="gray", linestyle="--", alpha=0.5)
ax.set_xlabel("Age")
ax.set_ylabel("Median Portfolio Value ($)")
ax.set_title("Stress Scenario Comparison")
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x/1e6:.1f}M" if x >= 1e6 else f"${x/1e3:.0f}K"))
plt.tight_layout()
plt.show()

## Part 3: Sensitivity Analysis (Tornado)

In [ ]:
from monteplan.analytics.sensitivity import run_sensitivity

report = run_sensitivity(
    plan=plan,
    market=market,
    policies=policies,
    sim_config=SimulationConfig(n_paths=1000, seed=42),
    perturbation_pct=0.10,
)

# Sort by absolute impact
sorted_results = sorted(report.results, key=lambda x: abs(x.impact), reverse=True)

for r in sorted_results[:8]:
    print(f"{r.parameter_name:25s}: {r.impact:+.1%} ({r.low_success:.1%} to {r.high_success:.1%})")

In [ ]:
# Tornado chart
top_n = min(10, len(sorted_results))
top = sorted_results[:top_n]

fig, ax = plt.subplots(figsize=(10, 6))
names = [r.parameter_name for r in reversed(top)]
low_delta = [r.low_success - report.base_success_probability for r in reversed(top)]
high_delta = [r.high_success - report.base_success_probability for r in reversed(top)]

y_pos = range(len(names))
ax.barh(y_pos, low_delta, color="#d62728", alpha=0.7, label="Low (-10%)")
ax.barh(y_pos, high_delta, color="#2ca02c", alpha=0.7, label="High (+10%)")
ax.set_yticks(y_pos)
ax.set_yticklabels(names)
ax.set_xlabel("Change in Success Probability")
ax.set_title("Sensitivity Tornado Chart")
ax.axvline(0, color="black", linewidth=0.5)
ax.legend()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:+.0%}"))
plt.tight_layout()
plt.show()

## Part 4: 2D Sensitivity Heatmap

In [ ]:
from monteplan.analytics.sensitivity import run_2d_sensitivity

heatmap = run_2d_sensitivity(
    plan=plan,
    market=market,
    policies=policies,
    sim_config=SimulationConfig(n_paths=1000, seed=42),
    x_param="Monthly Spending",
    y_param="US Stocks Return",
    x_range=(4000, 6000),
    y_range=(0.05, 0.09),
    x_steps=5,
    y_steps=5,
)

# Plot heatmap
grid = np.array(heatmap.success_grid)
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(grid, cmap="RdYlGn", aspect="auto", origin="lower",
               extent=[heatmap.x_values[0], heatmap.x_values[-1],
                       heatmap.y_values[0], heatmap.y_values[-1]])
ax.set_xlabel(heatmap.x_param_name)
ax.set_ylabel(heatmap.y_param_name)
ax.set_title("Success Probability Heatmap")
plt.colorbar(im, ax=ax, label="Success %", format="{x:.0%}")
ax.plot(heatmap.base_x_value, heatmap.base_y_value, "k*", markersize=15, label="Base case")
ax.legend()
plt.tight_layout()
plt.show()

## Part 5: Metrics

In [ ]:
from monteplan.analytics.metrics import compute_metrics, max_drawdown_distribution

# Need full paths for metrics
sim_with_paths = SimulationConfig(n_paths=1000, seed=42, store_paths=True)
result_paths = simulate(plan, market, policies, sim_with_paths)

if result_paths.all_paths is not None:
    retirement_step = (plan.retirement_age - plan.current_age) * 12
    metrics = compute_metrics(result_paths.all_paths, retirement_step)
    print(f"Success probability: {metrics.success_probability:.1%}")
    print(f"Median terminal wealth: ${metrics.terminal_wealth_p50:,.0f}")

    drawdowns = max_drawdown_distribution(result_paths.all_paths)
    print(f"\nMax drawdown distribution:")
    for k, v in drawdowns.items():
        print(f"  {k}: {v:.1%}")
else:
    print("store_paths=True required for per-path metrics")

## Part 6: Antithetic Variates

In [ ]:
# Compare standard vs antithetic variance
seeds = list(range(10))
standard_rates = []
anti_rates = []

for s in seeds:
    res_std = simulate(plan, market, policies, SimulationConfig(n_paths=1000, seed=s))
    res_anti = simulate(plan, market, policies, SimulationConfig(n_paths=1000, seed=s, antithetic=True))
    standard_rates.append(res_std.success_probability)
    anti_rates.append(res_anti.success_probability)

print(f"Standard -- Mean: {np.mean(standard_rates):.1%}, Std: {np.std(standard_rates):.2%}")
print(f"Antithetic -- Mean: {np.mean(anti_rates):.1%}, Std: {np.std(anti_rates):.2%}")
print(f"\nVariance reduction: {1 - np.var(anti_rates)/np.var(standard_rates):.0%}")

## Part 7: Reproducibility

In [ ]:
# Same seed = same results
r1 = simulate(plan, market, policies, SimulationConfig(n_paths=1000, seed=42))
r2 = simulate(plan, market, policies, SimulationConfig(n_paths=1000, seed=42))

print(f"Run 1: {r1.success_probability:.6f}")
print(f"Run 2: {r2.success_probability:.6f}")
print(f"Identical: {r1.success_probability == r2.success_probability}")
print(f"\nConfig hash: {r1.config_hash}")
print(f"Engine version: {r1.engine_version}")